В этом файле содержатся заготовки кода, который понадобится в дальнешей работе: набор библиотек, необходимых для запуска среды gym в гугл-коллабе + пример работы с одной из простейших сред для обучения CartPole (или "Обратный маятник") 

In [ ]:
!pip install gym pyvirtualdisplay atari-py > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg x11-utils > /dev/null 2>&1

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64

from IPython.display import HTML
from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

Агентом в среде CartPole является подвижный подвес для обратного маятника. Задача алгоритма управления заключается в том, чтобы смещая подвес вдоль оси X в рамках диапазона, удерживать маятник в вертикальном положении (отклонение маятника от оси Y не более 12 градусов). Ниже приведен код, позволяющий создать среду и немного порулить агентом с помощью случайной стратегии. 

Задача для размышления: проанализировать пространство действий и наблюдений для описанной игры. 
1. Простраство наблюдей доступно полностью или частично?
2. Пространство действий дискретное или непрерывное? 

In [ ]:
env = wrap_env(gym.make('CartPole-v0'))

In [ ]:
is_done = False
env.reset()
env.render()
while not is_done:
    action = env.action_space.sample()
    new_state, reward, is_done, info = env.step(action)
    print(new_state)
    env.render()

[-0.03540065  0.2335353  -0.03908474 -0.32639919]
[-0.03072995  0.42919129 -0.04561272 -0.63114703]
[-0.02214612  0.62491899 -0.05823567 -0.93783872]
[-0.00964774  0.43062855 -0.07699244 -0.66400872]
[-0.00103517  0.23665731 -0.09027261 -0.39652722]
[ 0.00369798  0.04292435 -0.09820316 -0.13361517]
[ 0.00455646  0.23930581 -0.10087546 -0.45559342]
[ 0.00934258  0.045744   -0.10998733 -0.19633382]
[ 0.01025746  0.24225327 -0.11391401 -0.52158773]
[ 0.01510253  0.43877886 -0.12434576 -0.84788386]
[ 0.0238781   0.63535754 -0.14130344 -1.17693873]
[ 0.03658525  0.83200341 -0.16484221 -1.51037277]
[ 0.05322532  0.63921758 -0.19504967 -1.27335848]
[ 0.06600967  0.44704413 -0.22051684 -1.04754597]


In [ ]:
show_video()

In [ ]:
n_episode = 10000
total_rewards = []
for episode in range(n_episode):
    state = env.reset()
    total_reward = 0
    is_done = False
    while not is_done:
        action = env.action_space.sample()
        state, reward, is_done, _ = env.step(action)
        total_reward += reward
    total_rewards.append(total_reward)


print('Average total reward over {} episodes: {}'.format(n_episode, sum(total_rewards) / n_episode))

Average total reward over 10000 episodes: 22.3791
